### 범주형(카테고리) 데이터 처리

#### 구간 분할

- 데이터 분석 알고리즘에 따라서는 연속 데이터를 그대로 사용하기 보다는 일정한 구간(bin)으로 나눠서 분석하는 것이 효율적인 경우가 있음
- 연속적인 값을 일정한 수준이나 정도를 나타내는 이산적인 값으로 나타내어 구간별 차이를 드러내는 것

- 구간분할 : 연속 변수를 일정한 구간으로 나누고, 각 구간을 범주형 이산 변수로 변환하는 과정
- 판다스 cut() 함수를 이용하면 연속 데이터를 여러 구간으로 나누고 범주형 데이터로 변환할 수 있음

In [4]:
# 5.10_data_binning.py

# 라이브러리 불러오기
import pandas as pd
import numpy as np

# read_csv() 함수로 df 생성
df = pd.read_csv('/Users/youju/Downloads/5674-833_4th/part5/auto-mpg.csv', header = None)

# 열 이름 지정
df.columns = ['mpg', 'cylinders', 'displacement', 'horsepower', 'weight',
              'acceleration', 'model year', 'origin', 'name']

# horepower 열의 누락 데이터('?')를 삭제하고 실수형으로 변환
df['horsepower'].replace('?', np.nan, inplace = True) # '?'을 np.nan으로 변경
df.dropna(subset = ['horsepower'], axis = 0, inplace = True) # 누락 데이터 행 삭제
df['horsepower'] = df['horsepower'].astype('float') # 문자열을 실수형으로 변환

# np.histogram 함수로 3개의 bin으로 구분할 경계값의 리스트 구하기
count, bin_dividers = np.histogram(df['horsepower'], bins = 3)
print(bin_dividers)

[ 46.         107.33333333 168.66666667 230.        ]


In [5]:
# 3개의 bin에 이름 저장
bin_names = ['저출력', '보통출력', '고출력']

# pd.cut 함수로 각 데이터를 3개의 bin에 할당
df['hp_bin'] = pd.cut(x = df['horsepower'], # 데이터 배열
                      bins = bin_dividers, # 경계값 리스트
                      labels = bin_names, # bin 이름
                      include_lowest = True) # 첫 경계값 포함

# horsepower 열, hp_bin 열의 첫 15행 출력
print(df[['horsepower', 'hp_bin']].head(15))

    horsepower hp_bin
0        130.0   보통출력
1        165.0   보통출력
2        150.0   보통출력
3        150.0   보통출력
4        140.0   보통출력
5        198.0    고출력
6        220.0    고출력
7        215.0    고출력
8        225.0    고출력
9        190.0    고출력
10       170.0    고출력
11       160.0   보통출력
12       150.0   보통출력
13       225.0    고출력
14        95.0    저출력


#### 더미 변수

- 카테고리를 나타내는 범주형 데이터를 회귀분석 등 머신러닝 알고리즘에 바로 사용할 수 없는 경우, 컴퓨터가 인식 가능한 입력값으로 변환해야 함
- 이런 경우 숫자 0 또는 1로 표현되는 더미 변수(dummy variable)를 사용
- 0과 1은 수의 크고 작음을 나타내지 않고, 어떤 특성(feature)이 있는지 없는지 여부를 표시
- 해당 특성이 존재하면 1로 표현, 존재하지 않으면 0으로 표현
- 범주형 데이터를 컴퓨터가 인식할 수 있도록 숫자 0과 1로만 구성되는 원핫벡터(one hot vector)로 변환한다고 해서 원핫인코딩(one-hot-encoding)이라고도 부름

In [6]:
# 5.11_get_dummies.py

# 생략 (예제 5.10과 동일)

# np.histogram 함수로 3개의 bin으로 구분할 경계깞의 리스트 구하기
count, bin_dividers = np.histogram(df['horsepower'], bins = 3)

# 3개의 bon에 이름 지정
bin_names = ['저출력', '보통출력', '고출력']

# pd.cut으로 각 데이터를 3개의 bin에 할당
df['hp_bin'] = pd.cut(x = df['horsepower'], # 데이터 배열
                      bins = bin_dividers, # 경계값 리스트
                      labels = bin_names, # bin 이름
                      include_lowest = True) # 첫 경계값 포함

# hp_bin 열의 범주형 데이터를 더미 변수로 변환
horsepower_dummies = pd.get_dummies(df['hp_bin'])
print(horsepower_dummies.head(15))


    저출력  보통출력  고출력
0     0     1    0
1     0     1    0
2     0     1    0
3     0     1    0
4     0     1    0
5     0     0    1
6     0     0    1
7     0     0    1
8     0     0    1
9     0     0    1
10    0     0    1
11    0     1    0
12    0     1    0
13    0     0    1
14    1     0    0


In [8]:
# 5.12_one_hot_encoding.py

# 생략 (5.11과 동일)

# sklern 라이브러리 불러오기
from sklearn import preprocessing

# 전처리를 위한 encoder 객체 만들기
label_encoder = preprocessing.LabelEncoder() # label encoder 생성
onehot_encoder = preprocessing.OneHotEncoder() # one hot encoder 생성

# label encoder로 문자열 범주를 숫자형 범주로 변환
onehot_labeled = label_encoder.fit_transform(df['hp_bin'].head(15))
print(onehot_labeled)
print(type(onehot_labeled))

# 2차원 행렬로 변경
onehot_reshaped = onehot_labeled.reshape(len(onehot_labeled), 1)
print(onehot_reshaped)
print(type(onehot_reshaped))

# 희소행렬로 변환
onehot_fitted = onehot_encoder.fit_transform(onehot_reshaped)
print(onehot_fitted)
print(type(onehot_fitted))

[1 1 1 1 1 0 0 0 0 0 0 1 1 0 2]
<class 'numpy.ndarray'>
[[1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [2]]
<class 'numpy.ndarray'>
  (0, 1)	1.0
  (1, 1)	1.0
  (2, 1)	1.0
  (3, 1)	1.0
  (4, 1)	1.0
  (5, 0)	1.0
  (6, 0)	1.0
  (7, 0)	1.0
  (8, 0)	1.0
  (9, 0)	1.0
  (10, 0)	1.0
  (11, 1)	1.0
  (12, 1)	1.0
  (13, 0)	1.0
  (14, 2)	1.0
<class 'scipy.sparse._csr.csr_matrix'>


### 정규화

- 각 변수(데이터프레임의 열)에 들어 있는 숫자 데이터의 상대적 크기 차이 때문에 머신러닝 분석 결과가 달라질 수 있음
- 숫자 데이터의 상대적 크기 차이를 제거할 필요가 있음
- 정규화(normalization) : 각 열(변수)에 속하는 데이터 값을 동일한 크기 기준으로 나눈 비율로 나타내는 것
- 정규화 과정을 거친 데이터의 범위는 0 ~ 1 또는 -1 ~ 1이 됨


- 각 열(변수)의 데이터를 해당 열의 최대값(의 절대값)으로 나누는 방법
- 어떤 열의 원소 값을 그 열의 최대값으로 나누면 가장 큰 값은 최대값 자기자신을 나눈 1

In [12]:
# 5.13_normalize_max.py

# 라이브러리 불러오기
import pandas as pd
import numpy as np

# read_csv() 함수로 df 생성
df = pd.read_csv('/Users/youju/Downloads/5674-833_4th/part5/auto-mpg.csv', header = None)

# 열 이름 지정
df.columns = ['mpg', 'cylinders', 'displacement', 'horsepower', 'weight',
              'acceleration', 'model year', 'origin', 'name']

# horepower 열의 누락 데이터('?')를 삭제하고 실수형으로 변환
df['horsepower'].replace('?', np.nan, inplace = True) # '?'을 np.nan으로 변경
df.dropna(subset = ['horsepower'], axis = 0, inplace = True) # 누락 데이터 행 삭제
df['horsepower'] = df['horsepower'].astype('float') # 문자열을 실수형으로 변환

# horsepower 열의 통계 요약 정보로 최대값(max) 확인
print(df.horsepower.describe())
print('\n')

# horsepower 열의 최대값의 절대값으로 모든 데이터를 나눠서 저장
df.horsepower = df.horsepower/abs(df.horsepower.max())

print(df.horsepower.head())
print('\n')
print(df.horsepower.describe())

count    392.000000
mean     104.469388
std       38.491160
min       46.000000
25%       75.000000
50%       93.500000
75%      126.000000
max      230.000000
Name: horsepower, dtype: float64


0    0.565217
1    0.717391
2    0.652174
3    0.652174
4    0.608696
Name: horsepower, dtype: float64


count    392.000000
mean       0.454215
std        0.167353
min        0.200000
25%        0.326087
50%        0.406522
75%        0.547826
max        1.000000
Name: horsepower, dtype: float64


- 각 열(변수)의 데이터 중에서 최대값(max)과 최소값(min)을 뺀 값으로 나누는 방법
- 각 열 데이터에서 해당 열의 최소값을 뺀 값을 분자로 하고, 해당 열의 최대값과 최소값의 차를 분모로 하는 수를 계산하면 가장 큰 값은 1

In [13]:
# 5.14_normalize_min_max.py

# 생략 (예제 5.13과 동일)

# horsepower 열의 통계 요약 정보로 최대값(max)과 최소값(min) 확인
print(df.horsepower.describe())
print('\n')

# horsepower 열의 최대값의 절대값으로 모든 데이터를 나눠서 저장
min_x = df.horsepower - df.horsepower.min()
min_max = df.horsepower.max() - df.horsepower.min()
df.horsepower = min_x/min_max

print(df.horsepower.head())
print('\n')
print(df.horsepower.describe())

count    392.000000
mean       0.454215
std        0.167353
min        0.200000
25%        0.326087
50%        0.406522
75%        0.547826
max        1.000000
Name: horsepower, dtype: float64


0    0.456522
1    0.646739
2    0.565217
3    0.565217
4    0.510870
Name: horsepower, dtype: float64


count    392.000000
mean       0.317768
std        0.209191
min        0.000000
25%        0.157609
50%        0.258152
75%        0.434783
max        1.000000
Name: horsepower, dtype: float64


### 시계열 데이터

- Timestamp : 특정한 시점을 기록
- Period : 두 시점 사이의 일정한 기간

#### 다른 자료형을 시계열 객체로 변환

- 우리가 접하는 많은 시간 데이터들은 별도의 시간 자료형(파이썬 datetime 라이브러리 등)으로 기록되지 않고, 문자열 또는 숫자로 저장되는 경우가 많음
- 판다스는 다른 자료형으로 저장된 시간 데이터를 판다스 시계열 객체인 Timestamp로 변환하는 함수를 제공

##### 문자열을 Timestamp로 변환

- 판다스 to_datetime() 함수를 사용하면 문자열 등 다른 자료형을 판다스 Timestamp를 나타내는 datetime64 자료형으로 변환 가능

In [14]:
# 5.15_time_series.py

# 라이브러리 불러오기
import pandas as pd

# read_csv() 함수로 csv 파일을 가져와서 df로 변환
df = pd.read_csv('/Users/youju/Downloads/5674-833_4th/part5/stock-data.csv')

# 데이터 내용 및 자료형 확인
print(df.head())
print('\n')
print(df.info())

         Date  Close  Start   High    Low  Volume
0  2018-07-02  10100  10850  10900  10000  137977
1  2018-06-29  10700  10550  10900   9990  170253
2  2018-06-28  10400  10900  10950  10150  155769
3  2018-06-27  10900  10800  11050  10500  133548
4  2018-06-26  10800  10900  11000  10700   63039


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Date    20 non-null     object
 1   Close   20 non-null     int64 
 2   Start   20 non-null     int64 
 3   High    20 non-null     int64 
 4   Low     20 non-null     int64 
 5   Volume  20 non-null     int64 
dtypes: int64(5), object(1)
memory usage: 1.1+ KB
None


In [15]:
# 문자열 데이터(시리즈 객체)를 판다스 Timestamp로 변환
df['new_Date'] = pd.to_datetime(df['Date']) # df에 새로운 열로 추가

# 데이터 내용 및 자료형 확인
print(df.head())
print('\n')
print(df.info())
print('\n')
print(type(df['new_Date'][0]))

         Date  Close  Start   High    Low  Volume   new_Date
0  2018-07-02  10100  10850  10900  10000  137977 2018-07-02
1  2018-06-29  10700  10550  10900   9990  170253 2018-06-29
2  2018-06-28  10400  10900  10950  10150  155769 2018-06-28
3  2018-06-27  10900  10800  11050  10500  133548 2018-06-27
4  2018-06-26  10800  10900  11000  10700   63039 2018-06-26


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Date      20 non-null     object        
 1   Close     20 non-null     int64         
 2   Start     20 non-null     int64         
 3   High      20 non-null     int64         
 4   Low       20 non-null     int64         
 5   Volume    20 non-null     int64         
 6   new_Date  20 non-null     datetime64[ns]
dtypes: datetime64[ns](1), int64(5), object(1)
memory usage: 1.2+ KB
None


<class 'pandas._libs.tslibs.timestamps.T

In [16]:
# 시계열 값으로 변환된 열을 새로운 행 인덱스로 지정. 기존 날짜 열은 삭제
df.set_index('new_Date', inplace = True)
df.drop('Date', axis = 1, inplace = True)

# 데이터 내용 및 자료형 확인
print(df.head())
print('\n')
print(df.info())

            Close  Start   High    Low  Volume
new_Date                                      
2018-07-02  10100  10850  10900  10000  137977
2018-06-29  10700  10550  10900   9990  170253
2018-06-28  10400  10900  10950  10150  155769
2018-06-27  10900  10800  11050  10500  133548
2018-06-26  10800  10900  11000  10700   63039


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 20 entries, 2018-07-02 to 2018-06-01
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   Close   20 non-null     int64
 1   Start   20 non-null     int64
 2   High    20 non-null     int64
 3   Low     20 non-null     int64
 4   Volume  20 non-null     int64
dtypes: int64(5)
memory usage: 960.0 bytes
None


##### Timestamp를 Period로 변환

- 판다스 to_period() 함수를 이용하면 일정한 기간을 나타내는 Period 객체로 Timestamp 객체를 변환할 수 있음
- freq 옵션에 기준이 되는 기간을 설정
1. 'D' -> 1일
2. 'M' -> 1개월
3. 'A' -> 1년. 1년이 끝나는 12월을 기준

In [17]:
# 5.16_time_series_period.py

# 라이브러리 불러오기
import pandas as pd

# 날짜 형식의 문자열로 구성되는 리스트 정의
dates = ['2019-01-01', '2020-03-01', '2021-06-01']

# 문자열의 배열(시리즈 객체)을 판다스 Timestamp로 변환
ts_dates = pd.to_datetime(dates)
print(ts_dates)
print('\n')

# Timestamp를 Period로 변환
pr_day = ts_dates.to_period(freq = 'D')
print(pr_day)
pr_month = ts_dates.to_period(freq = 'M')
print(pr_month)
pr_year = ts_dates.to_period(freq = 'A')
print(pr_year)

DatetimeIndex(['2019-01-01', '2020-03-01', '2021-06-01'], dtype='datetime64[ns]', freq=None)


PeriodIndex(['2019-01-01', '2020-03-01', '2021-06-01'], dtype='period[D]')
PeriodIndex(['2019-01', '2020-03', '2021-06'], dtype='period[M]')
PeriodIndex(['2019', '2020', '2021'], dtype='period[A-DEC]')


freq 옵션 종료
- D : day(1일)
- W : weel(1주)
- M : month end(월말)
- MS : month begin(월초)
- Q : quarter end(분기말)
- QS : qurter begin(분기초)
- A : year end(연말)
- AS : year begin(연초)
- B : busines day(휴일 제외)
- H : hour(1시간)
- T : minute(1분)
- S : second(1초)
- L : millisecind(1/1,000초)
- U : microsecind(1/1,000,000초)
- N : nanesecond(1/1,000,000,000초)

#### 시계열 데이터 만들기

##### Timestamp 배열

- 판다스 date_range() 함수를 사용하면 여러 개의 날짜(Timestamp)가 들어 있는 배열 형태의 시계열 데이터를 만들 수 있음
- 파이썬 range() 함수로 숫자 배열을 만드는 것과 비슷함

In [20]:
# 5.17_date_range.py

# 라이브러리 불러오기
import pandas as pd

# Timestamp의 배열 만들기 - 월 간격, 월의 시작일 기준
ts_ms = pd.date_range(start = '2019-01-01', # 날짜 범위 시작
                      end = None, # 날짜 범위 끝
                      periods = 6, # 생성할 Timestamp 개수
                      freq = 'MS', # 시간 간격(MS: 월의 시작일)
                      tz = 'Asia/Seoul') # 시간대(timezone)
print(ts_ms)

DatetimeIndex(['2019-01-01 00:00:00+09:00', '2019-02-01 00:00:00+09:00',
               '2019-03-01 00:00:00+09:00', '2019-04-01 00:00:00+09:00',
               '2019-05-01 00:00:00+09:00', '2019-06-01 00:00:00+09:00'],
              dtype='datetime64[ns, Asia/Seoul]', freq='MS')


In [21]:
# 월 간격, 월의 마지막 날 기준
ts_me = pd.date_range('2019-01-01', periods = 6,
                      freq = 'M', # 시간 간격(M: 월의 마지막 날)
                      tz = 'Asia/Seoul')
print(ts_me)
print('\n')

# 분기(3개월) 간격, 월의 마지막 날 기준
ts_3m = pd.date_range('2019-01-01', periods = 6,
                      freq = '3M', # 시간 간격(3M: 3개월)
                      tz = 'Asia/Seoul')
print(ts_3m)

DatetimeIndex(['2019-01-31 00:00:00+09:00', '2019-02-28 00:00:00+09:00',
               '2019-03-31 00:00:00+09:00', '2019-04-30 00:00:00+09:00',
               '2019-05-31 00:00:00+09:00', '2019-06-30 00:00:00+09:00'],
              dtype='datetime64[ns, Asia/Seoul]', freq='M')


DatetimeIndex(['2019-01-31 00:00:00+09:00', '2019-04-30 00:00:00+09:00',
               '2019-07-31 00:00:00+09:00', '2019-10-31 00:00:00+09:00',
               '2020-01-31 00:00:00+09:00', '2020-04-30 00:00:00+09:00'],
              dtype='datetime64[ns, Asia/Seoul]', freq='3M')


##### Period 배열

- 판다스 Period_range() 함수는 여러 개의 기간(Period)이 들어 있는 시계열 데이터를 만듦

In [22]:
# 5.18_period_range.py

# 라이브러리 불러오기
import pandas as pd

# Period 배열 만들기 - 1개월 길이
pr_m = pd.period_range(start = '2019-01-01',
                       end = None,
                       periods = 3,
                       freq = 'M') # 기간의 길이(M:월)
print(pr_m)

PeriodIndex(['2019-01', '2019-02', '2019-03'], dtype='period[M]')


In [23]:
# Period 배열 만들기 - 1시간 길이
pr_h = pd.period_range(start = '2019-01-01',
                       end = None,
                       periods = 3,
                       freq = 'H') # 기간의 길이(H: 시간)
print(pr_h)
print('\n')

# Period 배열 만들기 - 2시간 길이
pr_2h = pd.period_range(start = '2019-01-01',
                        end = None,
                        periods = 3,
                        freq = '2H') # 기간의 길이(2H: 2시간)
print(pr_2h)

PeriodIndex(['2019-01-01 00:00', '2019-01-01 01:00', '2019-01-01 02:00'], dtype='period[H]')


PeriodIndex(['2019-01-01 00:00', '2019-01-01 02:00', '2019-01-01 04:00'], dtype='period[2H]')


#### 시계열 데이터 활용

##### 날짜 데이터 분리

- 연-월-일 날짜 데이터에서 일부를 분리하여 추출 가능

In [26]:
# 5.19_datetime_format.py

# 라이브러리 불러오기
import pandas as pd

# read_csv() 함수로 파일 읽어와서 df로 변환
df = pd.read_csv('/Users/youju/Downloads/5674-833_4th/part5/stock-data.csv')

# 문자열인 날짜 데이터를 판다스 Timestamp로 변환
df['new_Date'] = pd.to_datetime(df['Date']) # df에 새로운 열로 추가
print(df.head())
print('\n')

# df 속성을 이용하여 new_Date 열의 연-월-일 정보를 년, 월, 일로 구분
df['Year'] = df['new_Date'].dt.year
df['Month'] = df['new_Date'].dt.month
df['Day'] = df['new_Date'].dt.day
print(df.head())

         Date  Close  Start   High    Low  Volume   new_Date
0  2018-07-02  10100  10850  10900  10000  137977 2018-07-02
1  2018-06-29  10700  10550  10900   9990  170253 2018-06-29
2  2018-06-28  10400  10900  10950  10150  155769 2018-06-28
3  2018-06-27  10900  10800  11050  10500  133548 2018-06-27
4  2018-06-26  10800  10900  11000  10700   63039 2018-06-26


         Date  Close  Start   High    Low  Volume   new_Date  Year  Month  Day
0  2018-07-02  10100  10850  10900  10000  137977 2018-07-02  2018      7    2
1  2018-06-29  10700  10550  10900   9990  170253 2018-06-29  2018      6   29
2  2018-06-28  10400  10900  10950  10150  155769 2018-06-28  2018      6   28
3  2018-06-27  10900  10800  11050  10500  133548 2018-06-27  2018      6   27
4  2018-06-26  10800  10900  11000  10700   63039 2018-06-26  2018      6   26


In [27]:
# Timestamp를 Period로 변환하여 연-월-일 표기 변경하기
df['Date_yr'] = df['new_Date'].dt.to_period(freq = 'A')
df['Date_m'] = df['new_Date'].dt.to_period(freq = 'M')
print(df.head())

         Date  Close  Start   High    Low  Volume   new_Date  Year  Month  \
0  2018-07-02  10100  10850  10900  10000  137977 2018-07-02  2018      7   
1  2018-06-29  10700  10550  10900   9990  170253 2018-06-29  2018      6   
2  2018-06-28  10400  10900  10950  10150  155769 2018-06-28  2018      6   
3  2018-06-27  10900  10800  11050  10500  133548 2018-06-27  2018      6   
4  2018-06-26  10800  10900  11000  10700   63039 2018-06-26  2018      6   

   Day Date_yr   Date_m  
0    2    2018  2018-07  
1   29    2018  2018-06  
2   28    2018  2018-06  
3   27    2018  2018-06  
4   26    2018  2018-06  


In [29]:
# 원하는 열을 새로운 행 인덱스로 지정
df.set_index('Date_m', inplace = True)
print(df.head())

               Date  Close  Start   High    Low  Volume   new_Date  Year  \
Date_m                                                                     
2018-07  2018-07-02  10100  10850  10900  10000  137977 2018-07-02  2018   
2018-06  2018-06-29  10700  10550  10900   9990  170253 2018-06-29  2018   
2018-06  2018-06-28  10400  10900  10950  10150  155769 2018-06-28  2018   
2018-06  2018-06-27  10900  10800  11050  10500  133548 2018-06-27  2018   
2018-06  2018-06-26  10800  10900  11000  10700   63039 2018-06-26  2018   

         Month  Day Date_yr  
Date_m                       
2018-07      7    2    2018  
2018-06      6   29    2018  
2018-06      6   28    2018  
2018-06      6   27    2018  
2018-06      6   26    2018  


##### 날짜 인덱스 활용

- Timestamp로 구성된 열을 행 인덱스로 지정하면 DatetimeIndex라는 고유 속성으로 변환됨
- Period로 구성된 열을 행 인덱스로 지정하면 PeriodIndex라는 속성을 가짐
- 날짜 인덱스를 활용하면 시계열 데이터에 대한 인덱싱과 슬라이싱이 편리

In [30]:
# 5.20_datetime_indexing.py

# 라이브러리 불러오기
import pandas as pd

# read_csv() 함수로 파일 읽어와 df로 변환
df = pd.read_csv('/Users/youju/Downloads/5674-833_4th/part5/stock-data.csv')

# 문자열인 날짜 데이터를 판다스 Timestamp로 변환
df['new_Date'] = pd.to_datetime(df['Date']) # 새로운 열에 추가
df.set_index('new_Date', inplace = True) # 행 인덱스로 지정

print(df.head())
print('\n')
print(df.index)

                  Date  Close  Start   High    Low  Volume
new_Date                                                  
2018-07-02  2018-07-02  10100  10850  10900  10000  137977
2018-06-29  2018-06-29  10700  10550  10900   9990  170253
2018-06-28  2018-06-28  10400  10900  10950  10150  155769
2018-06-27  2018-06-27  10900  10800  11050  10500  133548
2018-06-26  2018-06-26  10800  10900  11000  10700   63039


DatetimeIndex(['2018-07-02', '2018-06-29', '2018-06-28', '2018-06-27',
               '2018-06-26', '2018-06-25', '2018-06-22', '2018-06-21',
               '2018-06-20', '2018-06-19', '2018-06-18', '2018-06-15',
               '2018-06-14', '2018-06-12', '2018-06-11', '2018-06-08',
               '2018-06-07', '2018-06-05', '2018-06-04', '2018-06-01'],
              dtype='datetime64[ns]', name='new_Date', freq=None)


In [32]:
# 날짜 인덱스를 이용하여 데이터 선택하기
df_y = df['2018']
print(df_y.head())
print('\n')

df_ym = df.loc['2018-07'] # loc 인덱서 활용
print(df_ym)
print('\n')

df_ym_cols = df.loc['2018-07', 'Start' : 'High'] # 열 범위 슬라이싱
print(df_ym_cols)
print('\n')

df_ymd = df.loc['2018-07-02']
print(df_ymd)
print('\n')

df_ymd_range = df['2018-06-25' : '2018-06-20'] # 날짜 범위 지정
print(df_ymd_range)

                  Date  Close  Start   High    Low  Volume
new_Date                                                  
2018-07-02  2018-07-02  10100  10850  10900  10000  137977
2018-06-29  2018-06-29  10700  10550  10900   9990  170253
2018-06-28  2018-06-28  10400  10900  10950  10150  155769
2018-06-27  2018-06-27  10900  10800  11050  10500  133548
2018-06-26  2018-06-26  10800  10900  11000  10700   63039


                  Date  Close  Start   High    Low  Volume
new_Date                                                  
2018-07-02  2018-07-02  10100  10850  10900  10000  137977


            Start   High
new_Date                
2018-07-02  10850  10900


Date      2018-07-02
Close          10100
Start          10850
High           10900
Low            10000
Volume        137977
Name: 2018-07-02 00:00:00, dtype: object


Empty DataFrame
Columns: [Date, Close, Start, High, Low, Volume]
Index: []


/var/folders/t0/85xt099562dflgf6nvxdqgl00000gn/T/ipykernel_24421/608460385.py:2: FutureWarning: Indexing a DataFrame with a datetimelike index using a single string to slice the rows, like `frame[string]`, is deprecated and will be removed in a future version. Use `frame.loc[string]` instead.
  df_y = df['2018']


In [33]:
# 시간 간격 계산. 최근 180일~189일 사이의 값들만 선택
today = pd.to_datetime('2018-12-25') # 기준일 생성
df['time_delta'] = today - df.index # 날짜 차이 계산
df.set_index('time_delta', inplace = True) # 행 인덱스로 지정
df_180 = df['180 days' : '189 days']
print(df_180)

                  Date  Close  Start   High    Low  Volume
time_delta                                                
180 days    2018-06-28  10400  10900  10950  10150  155769
181 days    2018-06-27  10900  10800  11050  10500  133548
182 days    2018-06-26  10800  10900  11000  10700   63039
183 days    2018-06-25  11150  11400  11450  11000   55519
186 days    2018-06-22  11300  11250  11450  10750  134805
187 days    2018-06-21  11200  11350  11750  11200  133002
188 days    2018-06-20  11550  11200  11600  10900  308596
189 days    2018-06-19  11300  11850  11950  11300  180656
